In [1]:
import numpy as np
R = np.linspace(-0.5, 0.5, num=150)
Z = np.linspace(2.0, 2.5, num=120)
Phi = np.linspace(0.0, 2*np.pi, num=240)
Rm, Zm, Phim = np.meshgrid(R, Z, Phi, indexing='ij')
f = Rm**2+Rm*Zm+np.exp(Zm)+Phim**1

In [7]:
np.allclose( np.gradient(f, R, axis=-3, edge_order=2)[:,0,0], (2*Rm+Zm)[:,0,0] )

True

In [27]:

from functools import lru_cache
import numpy as np
class _FieldDifferenatiableRZ:
    def __init__(self, field: np.ndarray, R, Z, Phi) -> None:
        self._field = field # field of shape (field dimension, nR, nZ, nPhi)
        self._R = R
        self._Z = Z
        self._Phi = Phi
    
    @lru_cache
    def diff_RZ(self, nR:int, nZ:int):
        if nR == 0 and nZ == 0:
            return self._field.copy()
        elif nZ > 0:
            return np.gradient(self.diff_RZ(nR, nZ-1), self._Z, axis=-2, edge_order=2)
        elif nR > 0:
            return np.gradient(self.diff_RZ(nR-1, nZ), self._R, axis=-3, edge_order=2)
        else:
            raise ValueError("nR, nZ to differentiate in the R,Z axis shall be >= 0.")

    @lru_cache
    def diff_RZ_interpolator(self, nR:int, nZ:int):
        from scipy.interpolate import RegularGridInterpolator
        if nR > 0 and nZ > 0:
            return RegularGridInterpolator( 
                (self._R[nR:-nR], self._Z[nZ:-nZ], self._Phi), self.diff_RZ(nR, nZ)[...,nR:-nR, nZ:-nZ,:],
                method="linear", bounds_error=True )
        elif nR > 0 and nZ == 0:
            return RegularGridInterpolator( 
                (self._R[nR:-nR], self._Z, self._Phi), self.diff_RZ(nR, nZ)[...,nR:-nR, :,:],
                method="linear", bounds_error=True )
        elif nR == 0 and nZ > 0:
            return RegularGridInterpolator( 
                (self._R, self._Z[nZ:-nZ], self._Phi), self.diff_RZ(nR, nZ)[...,:, nZ:-nZ,:],
                method="linear", bounds_error=True )
        elif nR == 0 and nZ == 0:
            return RegularGridInterpolator( 
                (self._R, self._Z, self._Phi), self.diff_RZ(nR, nZ)[...,:, :,:],
                method="linear", bounds_error=True )
        else:
            raise ValueError("nR, nZ to differentiate in the R,Z axis shall be >= 0.")



In [28]:
RBRdBPhi = _FieldDifferenatiableRZ(f, R, Z, Phi)

In [31]:
RBRdBPhi.diff_RZ(0,3)[1,5:110,1]

array([ 7.54599705,  7.57776962,  7.60967597,  7.64171667,  7.67389227,
        7.70620334,  7.73865047,  7.77123421,  7.80395515,  7.83681386,
        7.86981092,  7.90294692,  7.93622244,  7.96963806,  8.00319438,
        8.03689199,  8.07073149,  8.10471347,  8.13883852,  8.17310727,
        8.2075203 ,  8.24207824,  8.27678167,  8.31163121,  8.34662751,
        8.38177117,  8.41706277,  8.45250297,  8.48809242,  8.52383172,
        8.55972147,  8.59576234,  8.63195497,  8.6683    ,  8.70479806,
        8.74144977,  8.77825581,  8.81521685,  8.85233351,  8.88960642,
        8.92703628,  8.96462376,  9.00236948,  9.04027415,  9.0783384 ,
        9.11656292,  9.15494839,  9.19349549,  9.23220489,  9.27107727,
        9.31011334,  9.34931375,  9.38867922,  9.42821044,  9.46790811,
        9.50777293,  9.5478056 ,  9.58800683,  9.62837732,  9.66891781,
        9.70962897,  9.75051156,  9.79156629,  9.83279387,  9.87419504,
        9.91577055,  9.95752111,  9.99944745, 10.0415503 , 10.08

In [30]:
RBRdBPhi.diff_RZ_interpolator(0,3)([[R[1], Z[i], Phi[1]]for i in range(5, 110)])

array([ 7.54599705,  7.57776962,  7.60967597,  7.64171667,  7.67389227,
        7.70620334,  7.73865047,  7.77123421,  7.80395515,  7.83681386,
        7.86981092,  7.90294692,  7.93622244,  7.96963806,  8.00319438,
        8.03689199,  8.07073149,  8.10471347,  8.13883852,  8.17310727,
        8.2075203 ,  8.24207824,  8.27678167,  8.31163121,  8.34662751,
        8.38177117,  8.41706277,  8.45250297,  8.48809242,  8.52383172,
        8.55972147,  8.59576234,  8.63195497,  8.6683    ,  8.70479806,
        8.74144977,  8.77825581,  8.81521685,  8.85233351,  8.88960642,
        8.92703628,  8.96462376,  9.00236948,  9.04027415,  9.0783384 ,
        9.11656292,  9.15494839,  9.19349549,  9.23220489,  9.27107727,
        9.31011334,  9.34931375,  9.38867922,  9.42821044,  9.46790811,
        9.50777293,  9.5478056 ,  9.58800683,  9.62837732,  9.66891781,
        9.70962897,  9.75051156,  9.79156629,  9.83279387,  9.87419504,
        9.91577055,  9.95752111,  9.99944745, 10.0415503 , 10.08